# Web Scraping Job Vacancies

## Introduction

In this project, we built a web scraper to extract job listings from a popular job search platform. We extracted job titles, companies, locations, job descriptions, and other relevant information. Currently, this code is only tailored to scrape jobs from LinkedIn website.

Here are the main steps that were taken in this project:

1. Setup our development environment
2. Understand the basics of web scraping
3. Analyze the website structure of our job search platform
4. Write the Python code to extract job data from our job search platform
5. Save the data to a CSV file
6. Test our web scraper and refine our code as needed

## Prerequisites

Before starting this project, we already have some **basic knowledge of Python programming and HTML structure**. In addition, the **following Python packages** is needed to run the code:

- selenium
- requests
- BeautifulSoup
- mtranslate
- langdetect
- pandas
- csv

We choosed Google Chrome feature of Selenium, so we also also need a **ChromeDriver executable** to run the code. To get the ChromeDriver executable, we need to download it from the official [ChromeDriver website](https://sites.google.com/chromium.org/driver/). 

Some of these packages may or may not have been installed in your local computer environment, however if you'd like to install additional packages that are not included in this environment or are working off platform you can install additional packages using `!pip install packagename` within a notebook cell such as:

- `!pip install selenium`
- `!pip install requests`
- `!pip install beautifulsoup4`

We also need an **url of LinkedIn public job search result web page** specificied [this webpage](https://www.linkedin.com/jobs/search?position=1&pageNum=0) in which you can also add any filter to it (i.e. [link](https://www.linkedin.com/jobs/search?keywords=Docent&location=Amsterdam%2C%20North%20Holland%2C%20Netherlands&geoId=102011674&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0) for Teachers in Amsterdam). With this public job search feature, you don't have to sign in to any LinkedIn account.

Last but not least, we analyzed the web page structure to get the **specific HTML elements and its attributes** that contain the information we want to extract from the web page.

## Code

The logical flow of the code is this:
1. Scrape all the LinkedIn job links from the list of jobs in the Linkedin public job search result web page
2. For each LinkedIn job link, scrape all the job details from the job web page, clean and format the data, structurize it into field and value format
3. Save the collected and structurized data into an external CSV file

These are the technical steps that we did in the code:
1. Import required libraries
2. Define helper functions
3. Scrape the web page (run through the logical flow)
4. Export collected data into CSV

<em>Note:</em> For debugging purposes, some print command are kept

### Step 1: Import Required Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from mtranslate import translate
from langdetect import detect

import pandas as pd
import requests
import time
import re

### Step 2: Define Helper Functions

We need two helper functions:
1. Formatting LinkedIn job descriptions and translating it into English based on the detected language
2. Get job details from each LinkedIn job links and structurize it into a Dataframe

In [2]:
def translate_job_description_text(text):
    # Remove "Show more" and "Show less" from the text
    text = re.sub(r"(Show more|Show less)", "", text)

    # Remove unused new lines around the text
    text = text.strip()

    # Check if the text is not in English
    if detect(text) != 'en':
        # If not in English, translate the text to English using mtranslate
        translated_text = translate(text, 'en', detect(text))
    else:
        # If the text is already in English, keep it as it is
        translated_text = text
    return translated_text

def get_job_details(url):
    # Send a GET request to the URL and retrieve the HTML content
    response = requests.get(url)
    html_content = response.content
    
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Initialize the job_details dictionary
    job_details = {}
    
    # Extract the job details from the HTML content
    job_title = soup.find('h1', class_='top-card-layout__title').text.strip()
    company = soup.find('span', class_='topcard__flavor').text.strip()
    location = soup.find('span', class_='topcard__flavor topcard__flavor--bullet').text.strip()
    posted_time_ago = soup.find('span', class_='posted-time-ago__text').text.strip()
    job_description = soup.find('div', class_='description__text').text.strip()
    
    try:
        applicants_count = soup.find('span', class_='num-applicants__caption').text.strip()
    except AttributeError:
        applicants_count = soup.find('figcaption', class_='num-applicants__caption').text.strip()
    
    job_criteria_list = soup.find('ul', class_='description__job-criteria-list')
    criteria_items = job_criteria_list.find_all('li')
    criteria_dict = {}
    for item in criteria_items:
        subheader = item.find('h3', class_='description__job-criteria-subheader').text.strip()
        criteria_text = item.find('span', class_='description__job-criteria-text').text.strip()
        criteria_dict[subheader] = criteria_text
        
    # Translate the job details to English and add the job details to the dictionary
    job_details['Job Title'] = job_title
    job_details['Company'] = company
    job_details['Location'] = location
    job_details['Posted Time Ago'] = posted_time_ago
    job_details['Applicants Count'] = applicants_count
    job_details['Link'] = url
    job_details['Job Description'] = translate_job_description_text(job_description)
    job_details.update({key: value for key, value in criteria_dict.items()})

    # Return the job details dictionary
    return job_details

### Step 3: Scrape the Web Page

In this part, we are implementing the logical flow explained above. For point 1 in getting the LinkedIn job links, we used Selenium because the web page need to be scrolled down and some buttons need to be clicked for the completed job list to appear. For point 2 in scraping job details from each link, we use BeautifulSoup because we only need to get some text from certain HTML elements.

<em>Note</em>: LinkedIn job search result web page only display maximum of 1000 job posts and one loaded "page" contains 25 job posts, so we limit the iteration of scrolling or button clicks to only 40.

#### Get all the job links

In [6]:
# Set up the WebDriver
driver = webdriver.Chrome('./chromedriver_mac64/chromedriver')

/var/folders/f7/9nt6p5v94dq118j8xf4_cbs40000gn/T/ipykernel_42683/2989348950.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver_mac64/chromedriver')


In [7]:
# Navigate to the LinkedIn page
# This one is a web page of English teachers ("docent engels" in Dutch) jobs around the area of Heenvliet, The Netherlands
driver.get('https://www.linkedin.com/jobs/search?keywords=Docent%20Engels&location=Heenvliet%2C%20South%20Holland%2C%20Netherlands&geoId=100029650&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0')

viewed_all_jobs_hidden = True # Initially the "Viewed all jobs" text is hidden
iteration_count = 0  # Initialize the counter
max_iterations = 40 # Maximum number of iterations


# Scroll down the page or click "See more jobs" button until the "Viewed all jobs" text is not hidden
while viewed_all_jobs_hidden and iteration_count < max_iterations:
    # Check if the "See more jobs" button is visible
    see_more_button = driver.find_elements(By.XPATH, "//button[contains(text(), 'See more jobs')]")
    see_more_button_visible = 'infinite-scroller__show-more-button--visible' in see_more_button[0].get_attribute('class')
    
    if see_more_button and not see_more_button_visible:
        # "See more jobs" button is not visible then scroll to the bottom of the page
        print("Scrolling..")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    else:
        # "See more jobs" button is visible then click the "See more jobs" button
        print("See More Button visible.. clicking.")
        see_more_button[0].click()
    
    # Wait for the page to load more content
    time.sleep(2)
    
    # Check if the "Viewed all jobs" text is still hidden
    viewed_all_jobs = driver.find_elements(By.XPATH, "//div[contains(@class, 'see-more-jobs__viewed-all')]")
    viewed_all_jobs_hidden = 'hidden' in viewed_all_jobs[0].get_attribute('class')
    # Increment the counter
    iteration_count += 1
print("Viewed all jobs or reached max iteration.\n")

# Get the page source
page_source = driver.page_source

# Replace "nl.linkedin.com" with "www.linkedin.com" in the page source
page_source = page_source.replace("nl.linkedin.com", "www.linkedin.com")

# Parse the HTML content
soup = BeautifulSoup(page_source, 'html.parser')

# Find all div elements with class "base-card"
div_elements = soup.find_all('div', class_='base-card')

# Extract the links from the div elements using list comprehension
links = [div.find('a')['href'].split('?refId=')[0] for div in div_elements]

# Find all anchor elements with class "base-card"
anchor_elements = soup.find_all('a', class_='base-card')

# Extract the links from the anchor elements using list comprehension and add to the list
links.extend(anchor['href'].split('?refId=')[0] for anchor in anchor_elements)

# Close the WebDriver
driver.quit()

Scrolling..
Viewed all jobs or reached max iteration.



#### Collect all job details

In [12]:
# Create an empty list to store the job details and erronous links
job_details_list = []
error_links = []

# Iterate over the links and retrieve job details
for i, link in enumerate(links, 1):
    try:
        job_details = get_job_details(link)
        job_details_list.append(job_details)
    except Exception as e:
        print(f"Error processing link: {link}")
        error_links.append(link)
        print(f"Error message: {str(e)}")
        continue
print("Done getting jobs details.")

# Create a dataframe from the job details list
df = pd.DataFrame(job_details_list)

Done getting jobs details.


In [7]:
# Check the dataframe
df

,Job Title,Company,Location,Posted Time Ago,Applicants Count,Link,Job Description,Seniority level,Employment type,Job function,Industries
0,"Leerkracht basisschool, Amsterdam",DIT IS WIJS,"Amsterdam, North Holland, Netherlands",3 days ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/leerkracht-...,Together we will talk and I will discover your...,Entry level,Temporary,Education,"Primary and Secondary Education, Higher Educat..."
1,"Leerkracht basisonderwijs, Amsterdam",DIT IS WIJS,"Amsterdam, North Holland, Netherlands",1 week ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/leerkracht-...,"As a teacher, you are compassionate, supportiv...",Entry level,Temporary,Education,"Primary and Secondary Education, Higher Educat..."
2,"Basisschool leerkracht, Amsterdam",DIT IS WIJS,"Amsterdam, North Holland, Netherlands",3 days ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/basisschool...,Together we are building on our mission: No mo...,Entry level,Temporary,Education,"Primary and Secondary Education, Higher Educat..."
3,"Docent beeldende vorming/tekenen ca. 0,6 fte",Voortgezet Onderwijs van Amsterdam,"Amsterdam, North Holland, Netherlands",1 week ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/docent-beel...,The Hyperion Lyceum is looking for a visual ar...,Not Applicable,Full-time,Education and Training,Education
4,Docent Spaans,Voortgezet Onderwijs van Amsterdam,"Amsterdam, North Holland, Netherlands",1 week ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/docent-spaa...,We are looking for a Spanish teacher for the n...,Not Applicable,Full-time,Education and Training,Education
...,...,...,...,...,...,...,...,...,...,...,...
313,"Groepsleerkracht groep 1/2 en groep 4, Amsterdam",Derec,"Amsterdam, North Holland, Netherlands",4 days ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/groepsleerk...,For this modern school I am looking for teache...,Entry level,Full-time,Education,"Primary and Secondary Education, Higher Educat..."
314,Groepsleerkracht groep 4 2023- 2024 zomer tot ...,Derec,"Amsterdam, North Holland, Netherlands",1 week ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/groepsleerk...,The vacancy is for a minimum of three days a w...,Entry level,Full-time,Education,"Primary and Secondary Education, Higher Educat..."
315,"Groepsleerkracht 2023-2024, Amsterdam",Derec,"Amsterdam, North Holland, Netherlands",3 days ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/groepsleerk...,This contemporary public primary school consis...,Entry level,Full-time,Education,"Primary and Secondary Education, Higher Educat..."
316,Tweedegraads docent Nederlands gezocht voor he...,Derec,"Amsterdam, North Holland, Netherlands",3 days ago,Be among the first 25 applicants,https://www.linkedin.com/jobs/view/tweedegraad...,As a teacher of Dutch you will be part of a cl...,Entry level,Full-time,Education,"Primary and Secondary Education, Higher Educat..."


### Step 4: Export Collected Data into CSV

In the code below, you can change the CSV file name according to your need. You can then open the CSV file and use the data for further analysis or for other needs.

In [8]:
df.to_csv('teachers_amsterdam_job_details.csv', index=False)

### Appendix: Random Tryout Codes

Here are some chuck of codes that we used to test out some functionalities or logic. We kept this codes displayed as a reminder that it is a good practice for a developer to test out some things separated from the main code to make sure things work they way we want to and to clear out assumptions. Also it's a reminder that programming is a constant learning journey :)

Testing out beautifulsoup find function

In [112]:
url = 'https://www.linkedin.com/jobs/view/docent-engels-basisonderwijs-0-2-fte-0-16-uur-at-nederland-vacature-3628419355'

# Send a GET request to the URL and retrieve the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

soup.find('span', class_='topcard__flavor').text.strip()

'Nederland Vacature Groep'

Try getting job details from one URL

In [181]:
# Test the function with a sample URL
# url = 'https://www.linkedin.com/jobs/view/docent-engels-basisonderwijs-0-2-fte-0-16-uur-at-nederland-vacature-3628419355'
# url = 'https://nl.linkedin.com/jobs/view/docent-engels-basisonderwijs-0-2-fte-rotterdam-at-dit-is-wijs-3633649514?refId=GqBRCejTBkfRh78d32r6Eg%3D%3D&trackingId=0IkmoS%2FaIROH%2BQYHO3b%2Bfw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'
url = 'https://www.linkedin.com/jobs/view/data-engineer-at-the-greenery-3595100903'
job_details = get_job_details(url)

# Print the job details
for key, value in job_details.items():
    print(key + ':', value)

nl
Job Title: Data Engineer
Company: The Greenery
Location: Barendrecht, South Holland, Netherlands
Posted Time Ago: 2 weeks ago
Applicants Count: 33 applicants
Link: https://www.linkedin.com/jobs/view/data-engineer-at-the-greenery-3595100903
Job Description: About us The Greenery supplies a complete and fresh range of fruit and vegetables all year round to international supermarket chains, wholesalers, caterers and the processing industry. Every day, together with our growers, we strive to allow consumers to enjoy fruit and vegetables carefree. The Greenery has been voted number 1 in the primary sector and one of the market leaders in fresh logistics services. Through continuous (product) innovation and focus on the digitization of our business processes, we are able to offer high-quality services. We do this, for example, for renowned supermarket chains such as Jumbo and PLUS, but also for a large number of wholesale customers. With this strategy, we are better placed than anyone els

Try translating job description

In [116]:
text = """Taken en verantwoordelijkhedenHallo! Ben jij de leuke leerkracht Engels die ik zoek?De kinderen van de groepen 6 t/m 8 maken graag kennis met de Engelse taal. Can you help them?Voor een basisschool in Rotterdam zoek ik een leerkracht Engels die graag de kinderen voorbereidt op een goede toekomst. Op deze basisschool werk je met interactieve lesmethodes die de kinderen stimuleert om iets nieuws te leren.Profiel kandidaatJe hebt een eerste- of tweedegraads bevoegdheid Engels;Je spreekt vloeiend Nederlands;Je hebt een talent voor het aanbrengen van structuur in de lessen en klassenorganisatie;Je hebt uitstekende pedagogische en didactische vaardigheden;Je hebt affiniteit met het basisonderwijs;Je wilt graag werken in regio Rotterdam.Geboden wordtNatuurlijk een marktconform salaris. Ben je zzp’er? Dan is het salaris afhankelijk van jouw kennis, kunde en ervaring;Met de auto of openbaar vervoer naar je werk? Geen probleem. Je krijgt van ons een reiskostenvergoeding van €0,21 per kilometer;Leuke collega’s waar je altijd mee kunt sparren;De mogelijkheid jezelf te ontwikkelen;Je krijgt toegang tot diverse opleidingsmogelijkheden en workshops;Een coach die er met de juiste ondersteuning en begeleiding voor zorgt dat je iedere dag met een grote glimlach naar je werk gaat.
      

        Show more

        


        Show less
"""

# Remove "Show more" and "Show less" from the text
text = re.sub(r"(Show more|Show less)", "", text)

# Remove unused new lines around the text
text = text.strip()

translator = Translator(from_lang="nl", to_lang="en")

# Split the text into smaller chunks
chunk_size = 500
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Translate each chunk
translated_text = ""
for chunk in chunks:
    translated_chunk = translator.translate(chunk)
    translated_text += translated_chunk

print(translated_text)

MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  21 HOURS 55 MINUTES 15 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MOREMYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  21 HOURS 55 MINUTES 15 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MOREMYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  21 HOURS 55 MINUTES 15 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE


In [122]:
from mtranslate import translate

text = """Taken en verantwoordelijkhedenHallo! Ben jij de leuke leerkracht Engels die ik zoek?De kinderen van de groepen 6 t/m 8 maken graag kennis met de Engelse taal. Can you help them?Voor een basisschool in Rotterdam zoek ik een leerkracht Engels die graag de kinderen voorbereidt op een goede toekomst. Op deze basisschool werk je met interactieve lesmethodes die de kinderen stimuleert om iets nieuws te leren.Profiel kandidaatJe hebt een eerste- of tweedegraads bevoegdheid Engels;Je spreekt vloeiend Nederlands;Je hebt een talent voor het aanbrengen van structuur in de lessen en klassenorganisatie;Je hebt uitstekende pedagogische en didactische vaardigheden;Je hebt affiniteit met het basisonderwijs;Je wilt graag werken in regio Rotterdam.Geboden wordtNatuurlijk een marktconform salaris. Ben je zzp’er? Dan is het salaris afhankelijk van jouw kennis, kunde en ervaring;Met de auto of openbaar vervoer naar je werk? Geen probleem. Je krijgt van ons een reiskostenvergoeding van €0,21 per kilometer;Leuke collega’s waar je altijd mee kunt sparren;De mogelijkheid jezelf te ontwikkelen;Je krijgt toegang tot diverse opleidingsmogelijkheden en workshops;Een coach die er met de juiste ondersteuning en begeleiding voor zorgt dat je iedere dag met een grote glimlach naar je werk gaat.
      

        Show more

        


        Show less
"""

# Remove "Show more" and "Show less" from the text
text = re.sub(r"(Show more|Show less)", "", text)

# Remove unused new lines around the text
text = text.strip()

translated_text = translate(text, "en", "nl")
print(translated_text)

Duties and ResponsibilitiesHello! Are you the nice English teacher I'm looking for? The children of groups 6 to 8 like to get acquainted with the English language. Can you help them? I am looking for an English teacher for a primary school in Rotterdam who likes to prepare the children for a good future. At this primary school you work with interactive teaching methods that stimulate the children to learn something new. Candidate profile You have a first or second level qualification in English; You speak fluent Dutch; have excellent pedagogical and didactic skills; You have an affinity with primary education; You would like to work in the Rotterdam region. Are you self-employed? Then the salary depends on your knowledge, skills and experience; Do you want to go to work by car or public transport? No problem. You will receive a travel allowance of € 0.21 per kilometer from us; Nice colleagues with whom you can always spar; The opportunity to develop yourself; You will have access to va

In [145]:
from mtranslate import translate
from langdetect import detect

text = "je t'aime tellement"
print(detect(text))

# Check if the text is not in English
if detect(text) != 'en':
    print("Translate the text to English using mtranslate")
    translated_text = translate(text, 'en', detect(text))
else:
    print("If the text is already in English, keep it as it is")
    translated_text = text

print(translated_text)

fr
Translate the text to English using mtranslate
I love you so much
